In [ ]:
import numpy as np
import scipy
import scipy.linalg
import inference

from matplotlib import pyplot as plt
from scipy.stats import ortho_group

In [ ]:
def test_random_projection(n=9):
    """Within an n dimensional space, project an orthogonal set of k vectors (a k-dimensional subspace) into a space of dimension n-k
    where a subspace of size nr_shared_vectors and of size 1 is shared"""

    nsamples = 100

    testdim = np.arange(1,n+1)
    meanerror = np.zeros(testdim.size)
    stderror = np.zeros(testdim.size)
    nr_shared_vectors = 3
    hier_list = [1,nr_shared_vectors,n]

    for (j, k) in enumerate(testdim):
        # create an orthogonal set of random vectors
        U = ortho_group.rvs(dim=n)
        U_rotated = U[:,1:nr_shared_vectors]
        U = U[:, :k]
#         print(U)
#         print(U_rotated[:,:nr_shared_vectors] )
        error = np.zeros(nsamples)
        if k==0:
            continue
        for i in range(nsamples):
            # create k orthogonal vectors
            V = ortho_group.rvs(dim=n)
            V = V[:, :k]
            U_rotated2 = U_rotated @ ortho_group.rvs(dim=nr_shared_vectors-1) 
            U_rotated2 = U_rotated2[:,:k]
            V[:,1:nr_shared_vectors] = U_rotated2[:,1:nr_shared_vectors] 
            V[:,0]= U[:,0]
            error[i] = calculate_proj_error(U, V, 'F')
        meanerror[j] = np.mean(error)
        stderror[j] = np.std(error)

    expected_error = expected_errors_random_projection_new(hier_list)
    expected_error_as_is = inference.expected_errors_random_projection(hier_list)
    plt.figure()
    plt.errorbar(testdim, meanerror, stderror,label="empirical")
    plt.errorbar(testdim, expected_error,label="updated calculation")
    plt.errorbar(testdim, expected_error_as_is, label="old calculation / paper")
    plt.legend()
                                                 

In [ ]:
def project_orthogonal_to(subspace_basis, vectors_to_project):
    """
    Subspace basis: linearly independent (not necessarily orthogonal or normalized)
    vectors that span the space orthogonal to which we want to project
    vectors_to_project: project these vectors into the orthogonal complement of the
    specified subspace
    """

    if not scipy.sparse.issparse(vectors_to_project):
        V = np.matrix(vectors_to_project)
    else:
        V = vectors_to_project

    orthogonal_proj = V - project_to(subspace_basis, V)

    return orthogonal_proj

def project_to(subspace_basis, vectors_to_project):
    """
    Subspace basis: linearly independent (not necessarily orthogonal or normalized)
    vectors that span the space to which we want to project
    vectors_to_project: project these vectors into the specified subspace
    """

    if not scipy.sparse.issparse(vectors_to_project):
        V = np.matrix(vectors_to_project)
    else:
        V = vectors_to_project

    if not scipy.sparse.issparse(subspace_basis):
        S = np.matrix(subspace_basis)
    else:
        S = subspace_basis

    # compute S*(S^T*S)^{-1}*S'*V
    X1 = S.T * V
    X2 = S.T * S
    projected = S * scipy.linalg.solve(X2, X1)

    return projected

def calculate_proj_error(U, V, norm):
    proj1 = project_orthogonal_to(U, V)
    error = scipy.linalg.norm(proj1)**2

    return error

def expected_errors_random_projection_new(levels):
    """
    Compute vector of expected errors for random projection
        dim_n -- ambient space
        levels of hierarchy [1, ..., dim_n]
    """
    start_end_pairs = zip(levels[:-1], levels[1:])

    expected_error = []
    for i, j in start_end_pairs:
        Ks = np.arange(i, j)
        errors = (Ks - i) * (j - Ks) / levels[-1]
        expected_error = np.hstack([expected_error, errors])
    expected_error = np.hstack([expected_error, 0])
    return expected_error

In [ ]:
test_random_projection()